<a href="https://colab.research.google.com/github/RaghavSrivastava25/Neural-Machine-Translation/blob/main/Eng_To_Esp_Translation_with_a_S2S_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [ ]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"


2654208/2638744 [==============================] - 0s 0us/step


In [ ]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

In [ ]:
for _ in range(10):
    print(random.choice(text_pairs))

("I don't have any children.", '[start] No tengo hijos. [end]')
("You didn't have to wake me up.", '[start] No hacía falta que me despertaras. [end]')
('She is a quiet woman.', '[start] Ella es una mujer tranquila. [end]')
("The pie's delicious.", '[start] El pastel está exquisito. [end]')
('I asked Tom to stay.', '[start] Le pedí a Tom que se quedara. [end]')
('The world is full of gossipers and liars.', '[start] El mundo está lleno de chusmas y mentirosos. [end]')
("I'll work for you as long as you keep paying me this well.", '[start] Trabajaré para ti mientras me sigas pagando así de bien. [end]')
('Tom is getting married next month.', '[start] Tom se va a casar el próximo mes. [end]')
("I'm not as rich as Tom.", '[start] No soy tan rico como Tom. [end]')
('He promised not to tell that to anyone.', '[start] Él prometió no decirle eso a nadie. [end]')


In [ ]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


In [ ]:
vocab_size = 15000
sequence_length = 20
batch_size = 64

eng_vectorization = TextVectorization(max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length)

spa_vectorization = TextVectorization(max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length + 1)

train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]

eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)